# OpenMined Public Key Infastructure

## Practical Tutorial Using Aries Agents

Welcome, if you have have run this notebook through the `./manage start` command then you have already overcome one of the most challenging aspects of this series. Congratulations!

In this tutorial series we are going to walk through how you can use use the AriesAgentController class from the aries-basic-controller package to interact with ACA-Py agent instances to establish connections, send messages and issue and verify credentials. 

The tutorial models three SSI enabled applications each running their own aries agents, you will use jupyter notebooks like this one to simulate both sides of the peer to peer interactions. Keep in mind the mental model for developing applications using aries - **Almost all interesting flows involve the exchange of messages, the interaction, between two entities as they invoke protocols.**

The actors in this scenario are as follows:

* **OM Authority** - The OpenMined authority is trusted to issue credentials to both data scientists and data owners. The authority acts as a gatekeeper, mediating trust between the data scientist and the data owner.
* **Data Scientist** - Data scientists demonstrate they are trustworthy to the OM Authority to receive a credential. The data scientist presents this credential to the data owner. They also request the data owner prove they also have a credential from the OM Authority.
* **Data Owner** - A data owner registers with the OM Authority, recieving a credential which they can present to a data scientist. They also challenge data scientists to prove they have a credential from the OM Authority.

Now, this scenario is made up and most likely the authority trusted to by data scientists and data owners to verify the other would be dependent on the context of the information flows being mediated. After taking this tutorial series you will be able to construct your own context-specific a trust domain specifying any number of trusted authorities and verifiable information exchanges that govern machine learning flows you care about.

Finally, we will show you how this authentication mechanism can be integrated into Duet.


## Docker Setup

Before going any further take a look at the `manage` and `docker-compose.yml` files in the root of the [om_pki_lesson](https://github.com/OpenMined/PyDentity/tree/master/tutorials/om_pki_lesson) folder and try to understand what is going on. 

The `./manage start` command you used created a series of docker containers defined in the `docker-compose.yml` file with variables specified within the `manage` script. These variables specify the ports and other configuration properties mostly used by ACA-Py agents.

If you type `docker ps` in a terminal you should see the list of all containers currently running on your machine.


### SSI Applications

Each SSI Application (OM Authority, Data Scientist, Data Owner) has three docker containers. An agent, notebook and a ngrok container. Remember if this were a production system these applications would all exist on distinct networks.

Let's review the OM Authority in detail.

#### om-authority-agent

All agents use one of the dockerfiles in the `dockerfiles/agents` folder. To configure and start an agent, a set of arguments must be specified.

These arguments specify a port for receiving messages from other agents - HTTP_PORT, a url to recieve input from a controller - ADMIN_URL, and a url for the agent to post webhook's too WEBHOOK_URL. There are a number of other configuration arguments available including specifying the public ledger the agent should use, the type of wallet and the seed for it. These are covered in detail in this [lab](https://github.com/cloudcompass/ToIPLabs/blob/master/docs/LFS173x/ACA-PyStartup.md) that is part of an EdX course on Hyperledger Aries. 

We use a common pattern to configure agents throughout this repo. The `manage` script within a tutorial or project defines the unique arguments for each agent. Within the `docker-compose` file these arguments are then mapped to generic argument names e.g. `HTTP_PORT:${OM_AUTHORITY_HTTP_PORT}`. The arguments are then used within another yml file which is passed into the agent on startup. These configuration yml files can be found in the `configuration` folder.

The following command is used to populate the yml file and start the aca-py agent using this configuration file.

```Docker
 command: [
        "-c",
          "python3 scripts/parse_yml_env_variables.py -c ./configuration/aries-args-basic.yaml; \
          sleep 20; \
          aca-py start --arg-file /tmp/agent_conf.yml"
      ]
```



#### om-authority-notebook

This container is created from the `dockerfiles/controllers/Dockerfile.basiccontroller`. It installs the aries-basic-controller package from the files within this directory. See lines 11 - 17. Then it initialises a juypter notebook service that is looking for notebooks in the workspace directory of the container. This directory is populated by mounting a volume in the `docker-compose.yml`. For example om-authority-notebook mounts the notebooks under `notebooks/om_authority` (this is one of them).

It is worth pointing out the all code within these notebooks is acting the business logic of the SSI application.

#### ngrok-om-authority

This container is used to expose your agent to the outside world. Ngrok exposes local servers behind NATs and firewalls to the public internet over secure tunnels. Visit [ngrok](https://ngrok.com/) for more information.

We use ngrok to expose the port that agent's receive messages from other agents to the public internet. It means you can use this agent to interact with any other agent out in the wild. For example one running on your phone as we will walk through in a later notebook.

One caveat of this, however, is that connections made from your docker agent will expire within 8 hours as a limitation of the ngrok free-tier. Therefore, these setups are intended for demonstration purposes only and should not be relied upon as is for production environments.


## Configuring the agent to point to the Sovrin StagingNet

The ledger the agent uses is an important part of the configuration process. An agent will attempt to resolve public DID's, credential definitions and schemas from this location, so if the other agent they interact with is using a different ledger then they will no longer be able to exchange verifiable information.

In this series we will use the Sovrin StagingNet, a public test network hosted by the Sovrin Foundation. You can view a transaction explorer for this network over at [IndyScan](https://indyscan.io/txs/SOVRIN_STAGINGNET/domain). We will be using IndyScan throughout this series to see the public information your OM Authority agent writes to the ledger.

To configure an ACA-Py agent to point at a specific network you must set the `--genesis-url` parameter on startup. We have taken care of this for you in the `manage` and `docker-compose.yml` setup. See the GENESIS_URL variable.



## Continue to Part 2

Great, now you have an idea about how we use Docker to manage our development environment and run the different SSI applications we will use throughout this series it is time to learn about using the AriesAgentController class from aries-basic-controller pip package to interact with an ACA-Py agent. 